<a href="https://colab.research.google.com/github/JosephMarks/AdviceGenerator/blob/main/023_converting_scanned_PDFs_to_excel_for_accounting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!apt-get install ocrmypdf -q

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts icc-profiles-free
  libart-2.0-2 libcupsfilters1 libcupsimage2 libgs9 libgs9-common libijs-0.35
  libjbig2dec0 libqpdf21 poppler-data python3-cffi-backend python3-defusedxml
  python3-img2pdf python3-olefile python3-pil python3-pkg-resources
  python3-pypdf2 python3-renderpm python3-reportlab python3-reportlab-accel
  python3-ruffus qpdf tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
  unpaper
Suggested packages:
  fonts-noto ghostscript-x ocrmypdf-doc python-watchdog img2pdf poppler-utils
  fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic
  | fonts-ipafont-gothic fonts-arphic-ukai fonts-arphic-uming fonts-nanum
  python3-pdfrw python-pil-doc

In [5]:
!pip install pdfplumber -q

In [6]:
!pip install airtable-python-wrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import os
from airtable import Airtable
import requests
import pdfplumber

In [8]:
def download_file(url):
    local_filename = url.split('/')[-1]
    
    with requests.get(url) as r:
        assert r.status_code == 200, f'error, status code is {r.status_code}'
        with open(local_filename, 'wb') as f:
            f.write(r.content)
        
    return local_filename

In [22]:
invoice = 'Novo.pdf'
# invoice_pdf = download_file(invoice)

In [10]:
invoice_pdf

'AcastStoriesIncW-9.pdf'

In [11]:
with pdfplumber.open(invoice_pdf) as pdf:
    page = pdf.pages[0]
    text = page.extract_text()
    # print(text)

In [55]:
os.system(f'ocrmypdf {invoice_pdf} output.pdf')

32512

In [57]:
with pdfplumber.open('output.pdf') as pdf:
    page = pdf.pages[0]
    text = page.extract_text(x_tolerance=2)
    print(text)

FileNotFoundError: ignored

In [23]:
lines = text.split('\n')

In [24]:
magicString = 'Number To Give the Requester for guidelines on whose number to enter.'
index = lines.index(magicString)
einNumber = index+1
inputValue = lines[einNumber].replace(" ","")
print(inputValue)

ValueError: ignored

In [15]:
# Base key, table name and api key defined for Airtable wrapper function
base_key = 'appLahV1A9j5NPq3D'
table_name = 'Test Vendors'
api_key = 'keyahhzaPENx3KKNR'

airtable = Airtable(base_key, table_name, api_key)


In [16]:
records = airtable.get_all()

In [143]:
# records

In [17]:
for record in records:
  try:
    url = record['fields']['W9 - W8BEN'][0]['url']
    invoice = url
    invoice_pdf = download_file(invoice)
    with pdfplumber.open(invoice_pdf) as pdf:
      page = pdf.pages[0]
      text = page.extract_text()
      lines = text.split('\n')
      magicString = 'Number To Give the Requester for guidelines on whose number to enter.'
      index = lines.index(magicString)
      einNumber = index+1
      inputValue = lines[einNumber].replace(" ","")
      fields = {'Tax Number': inputValue}
      airtable.update(record['id'], fields)
  except:
    continue


In [60]:
import re

In [ ]:
amt_re = re.compile(r'\.\d\d$')

In [ ]:
subt = 0

for line in lines:
    if 'SUBTOTAL' in line:
        break
    if amt_re.search(line):
        subt += float(line.split()[-1].replace(',', '').replace('$', ''))

In [ ]:
subt

2090.0

In [ ]:
lines

['Stanford Plumbing & Heating INVOICE',
 '123 Madison drive, Seattle, WA, 7829Q',
 'plumbingstanford.com',
 '990-120-4560',
 'BILL TO Invoice No: winvo2081',
 'Allen Smith Invoice Date: nae',
 '87Private st, Seattle, WA ‘Due Date: 12/01/18',
 'allen@gmailcom',
 '990-302-1898',
 'DESCRIPTION QTYHR UNITPRICE TOTAL',
 'Installed new kitchen sink(hours) 150.00',
 'Toto sink 1 $00.00 $00.00',
 'Worcester greenstar magnetic system filter 1 190.00 190.00',
 'Nest smart thermostat 1 250.00 250.00',
 'Worcester Greenstar 301 1 1500.00 1500.00',
 '‘SUBTOTAL 2590.00',
 'DISCOUNT 50.00',
 '‘SUBTOTAL LESS DISCOUNT 2540.00',
 'Tank you for your business!',
 'TAXRATE 12.00%',
 'TOTAL TAX 304.80',
 'Balance Due $2,844.80',
 'Terms & Instructions',
 'Please pay within 20 days by PayPal (bob@stanfordplumbingcom)',
 'Installed products have S year warranty.']